In [1]:
import tensorflow as tf
print(tf.__version__)

2.16.1


In [3]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    for gpu in gpus:
        tf.config.experimental.set_memory_growth(gpu, True)
        print("当前可用的GPU设备:", gpu)  # 输出当前的GPU设备
        # tf.config.set_visible_devices(gpu, 'GPU')
else:
    print("没有检测到可用的GPU设备。")


当前可用的GPU设备: PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')
当前可用的GPU设备: PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')


# 卷积神经网络（LeNet）

在[“多层感知机的从零开始实现”](../chapter_deep-learning-basics/mlp-scratch.ipynb)一节里我们构造了一个含单隐藏层的多层感知机模型来对Fashion-MNIST数据集中的图像进行分类。每张图像高和宽均是28像素。我们将图像中的像素逐行展开，得到长度为784的向量，并输入进全连接层中。然而，这种分类方法有一定的局限性。

1. 图像在同一列邻近的像素在这个向量中可能相距较远。它们构成的模式可能难以被模型识别。
2. 对于大尺寸的输入图像，使用全连接层容易导致模型过大。假设输入是高和宽均为$1,000$像素的彩色照片（含3个通道）。即使全连接层输出个数仍是256，该层权重参数的形状也是$3,000,000\times 256$：它占用了大约3 GB的内存或显存。这会带来过于复杂的模型和过高的存储开销。

卷积层尝试解决这两个问题。一方面，卷积层保留输入形状，使图像的像素在高和宽两个方向上的相关性均可能被有效识别；另一方面，卷积层通过滑动窗口将同一卷积核与不同位置的输入重复计算，从而避免参数尺寸过大。

卷积神经网络就是含卷积层的网络。本节里我们将介绍一个早期用来识别手写数字图像的卷积神经网络：LeNet [1]。这个名字来源于LeNet论文的第一作者Yann LeCun。LeNet展示了通过梯度下降训练卷积神经网络可以达到手写数字识别在当时最先进的结果。这个奠基性的工作第一次将卷积神经网络推上舞台，为世人所知。

## 5.5.1 LeNet模型

LeNet分为卷积层块和全连接层块两个部分。下面我们分别介绍这两个模块。

卷积层块里的基本单位是卷积层后接最大池化层：卷积层用来识别图像里的空间模式，如线条和物体局部，之后的最大池化层则用来降低卷积层对位置的敏感性。卷积层块由两个这样的基本单位重复堆叠构成。在卷积层块中，每个卷积层都使用$5\times 5$的窗口，并在输出上使用sigmoid激活函数。第一个卷积层输出通道数为6，第二个卷积层输出通道数则增加到16。这是因为第二个卷积层比第一个卷积层的输入的高和宽要小，所以增加输出通道使两个卷积层的参数尺寸类似。卷积层块的两个最大池化层的窗口形状均为$2\times 2$，且步幅为2。由于池化窗口与步幅形状相同，池化窗口在输入上每次滑动所覆盖的区域互不重叠。

卷积层块的输出形状为(批量大小, 通道, 高, 宽)。当卷积层块的输出传入全连接层块时，全连接层块会将小批量中每个样本变平（flatten）。也就是说，全连接层的输入形状将变成二维，其中第一维是小批量中的样本，第二维是每个样本变平后的向量表示，且向量长度为通道、高和宽的乘积。全连接层块含3个全连接层。它们的输出个数分别是120、84和10，其中10为输出的类别个数。

下面我们通过`Sequential`类来实现LeNet模型。

1. **导入必要组件**：
```python
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Flatten, Dense, Input
```
2. **网络结构定义**：
```python
net = Sequential([
    Input(shape=(28, 28, 1)),  # 输入层：28x28的灰度图像
```
3. **卷积块1**：
```python
    Conv2D(filters=6, kernel_size=5, activation='sigmoid'),  # 第一个卷积层
    MaxPool2D(pool_size=2, strides=2),  # 第一个池化层
```
特点：
- 6个5×5的卷积核
- 2×2最大池化，步幅为2
- sigmoid激活函数

4. **卷积块2**：
```python
    Conv2D(filters=16, kernel_size=5, activation='sigmoid'),  # 第二个卷积层
    MaxPool2D(pool_size=2, strides=2),  # 第二个池化层
```
特点：
- 16个5×5的卷积核
- 2×2最大池化，步幅为2
- sigmoid激活函数

5. **全连接层**：
```python
    Flatten(),  # 展平层
    Dense(120, activation='sigmoid'),  # 第一个全连接层
    Dense(84, activation='sigmoid'),   # 第二个全连接层
    Dense(10, activation='sigmoid')    # 输出层
```
特点：
- 展平卷积特征
- 三层全连接网络
- 最终输出10个类别

网络特点：
1. **层次结构**：
   - 2个卷积-池化块
   - 3个全连接层
   - 经典的LeNet-5架构

2. **激活函数**：
   - 全部使用sigmoid激活
   - 符合历史版本设计

3. **应用场景**：
   - 手写数字识别
   - 简单图像分类
   - 基础CNN教学

这个网络是深度学习历史上的重要里程碑：
- 首个成功的CNN架构之一
- 展示了CNN的基本组件
- 为现代CNN奠定了基础

注意：现代实现可能会：
- 使用ReLU激活函数
- 添加批量归一化
- 使用更现代的优化器

LeNet-5每一层的参数数量：

1. **第一个卷积层**：
```python
Conv2D(filters=6, kernel_size=5, activation='sigmoid')
```
参数计算：
- 卷积核参数：5 × 5 × 1 × 6 = 150
  - 5×5是卷积核大小
  - 1是输入通道数
  - 6是输出通道数（filters）
- 偏置参数：6（每个卷积核一个）
- 总参数：156

2. **第一个池化层**：
```python
MaxPool2D(pool_size=2, strides=2)
```
- 参数数量：0（池化层没有可训练参数）

3. **第二个卷积层**：
```python
Conv2D(filters=16, kernel_size=5, activation='sigmoid')
```
参数计算：
- 卷积核参数：5 × 5 × 6 × 16 = 2400
  - 5×5是卷积核大小
  - 6是输入通道数（上一层的filters）
  - 16是输出通道数
- 偏置参数：16
- 总参数：2416

4. **第二个池化层**：
```python
MaxPool2D(pool_size=2, strides=2)
```
- 参数数量：0

5. **展平层**：
```python
Flatten()
```
- 参数数量：0（只改变形状，不涉及计算）

6. **第一个全连接层**：
```python
Dense(120, activation='sigmoid')
```
参数计算：
- 输入维度：16 × 4 × 4 = 256（上一层展平后的大小）
- 权重参数：256 × 120 = 30720
- 偏置参数：120
- 总参数：30840

7. **第二个全连接层**：
```python
Dense(84, activation='sigmoid')
```
参数计算：
- 权重参数：120 × 84 = 10080
- 偏置参数：84
- 总参数：10164

8. **输出层**：
```python
Dense(10, activation='sigmoid')
```
参数计算：
- 权重参数：84 × 10 = 840
- 偏置参数：10
- 总参数：850

总参数统计：
- 第一卷积层：156
- 第二卷积层：2,416
- 第一全连接层：30,840
- 第二全连接层：10,164
- 输出层：850
- 总计：44,426个参数

观察：
1. 参数主要集中在全连接层
2. 卷积层参数相对较少
3. 池化层和展平层无参数

这种参数分布是典型的CNN特征：
- 卷积层通过参数共享减少参数量
- 全连接层往往占据大部分参数
- 总参数量相对现代网络较少

In [4]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Flatten, Dense, Input
net = Sequential([
    Input(shape=(28, 28, 1)),
    Conv2D(filters=6, kernel_size=5, activation='sigmoid'),
    MaxPool2D(pool_size=2, strides=2),
    Conv2D(filters=16, kernel_size=5, activation='sigmoid'),
    MaxPool2D(pool_size=2, strides=2),
    Flatten(),
    Dense(120, activation='sigmoid'),
    Dense(84, activation='sigmoid'),
    Dense(10, activation='sigmoid')
])

接下来我们构造一个高和宽均为28的单通道数据样本，并逐层进行前向计算来查看每个层的输出形状。

In [5]:
X = tf.random.uniform((1,28,28,1))
for layer in net.layers:
    X = layer(X)
    print(layer.name, 'output shape\t', X.shape)

conv2d output shape	 (1, 24, 24, 6)
max_pooling2d output shape	 (1, 12, 12, 6)
conv2d_1 output shape	 (1, 8, 8, 16)
max_pooling2d_1 output shape	 (1, 4, 4, 16)
flatten output shape	 (1, 256)
dense output shape	 (1, 120)
dense_1 output shape	 (1, 84)
dense_2 output shape	 (1, 10)


可以看到，在卷积层块中输入的高和宽在逐层减小。卷积层由于使用高和宽均为5的卷积核，从而将高和宽分别减小4，而池化层则将高和宽减半，但通道数则从1增加到16。全连接层则逐层减少输出个数，直到变成图像的类别数10。


## 5.5.2 获取数据和训练模型

下面我们来实验LeNet模型。实验中，我们仍然使用Fashion-MNIST作为训练数据集。

In [6]:
from tensorflow.keras.datasets import fashion_mnist

(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

29515/29515 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
26421880/26421880 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
5148/5148 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
4422102/4422102 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [7]:
train_images = tf.reshape(train_images, 
                          (train_images.shape[0],
                           train_images.shape[1],
                           train_images.shape[2], 1))
print(train_images.shape)

test_images = tf.reshape(test_images, 
                         (test_images.shape[0],
                          test_images.shape[1],
                          test_images.shape[2], 1))


(60000, 28, 28, 1)


损失函数和训练算法依然采用交叉熵损失函数(cross entropy)和小批量随机梯度下降(SGD)

这段代码在配置神经网络的训练参数,我用中文为您解释主要内容:

1. **优化器设置**:
- 使用随机梯度下降(SGD)优化器
- 学习率设为0.9
- 动量参数设为0
- 没有使用Nesterov加速梯度

2. **模型编译配置**:
- 使用`compile()`方法配置模型训练参数
- 损失函数选择`sparse_categorical_crossentropy`(稀疏分类交叉熵),适用于整数标签的多分类问题
- 评估指标选择准确率(accuracy)

这是训练深度学习模型前的标准配置步骤。值得注意的是:
- 学习率0.9相对较大,可能会导致训练不稳定
- 没有使用动量,这可能会使收敛速度较慢
- 这些参数设置比较基础,现代实践中可能会选择更复杂的优化器(如Adam)或添加正则化等技术

SGD优化器中的momentum和nesterov参数：

### Momentum (动量)
- **作用**：帮助优化器在训练过程中保持一定的"惯性"，可以加快收敛速度并避免陷入局部最小值
- **工作原理**：
  - 不仅考虑当前梯度，还会考虑之前累积的梯度方向
  - momentum=0.0 表示完全不使用动量
  - 通常设置为0.9左右会有较好效果
- **优势**：
  - 可以加快收敛速度
  - 有助于跳出局部最小值
  - 在梯度方向震荡时具有平滑效果

### Nesterov (涅斯捷罗夫加速梯度)
- **作用**：是动量方法的一种改进版本
- **工作原理**：
  - nesterov=False 表示不使用这种改进
  - 当设置为True时，会先根据动量移动，然后再计算梯度
  - 相比普通动量方法，能提供更好的收敛性能
- **优势**：
  - 对梯度提供了一种"前瞻性"的修正
  - 通常比普通动量方法收敛更快
  - 在某些情况下能获得更好的训练效果

### 建议的改进
当前代码中这两个参数都没有启用（momentum=0.0，nesterov=False），建议的改进方案：
```python
optimizer = SGD(
    learning_rate=0.9,
    momentum=0.9,    # 添加动量
    nesterov=True    # 启用Nesterov加速
)
```

这样的设置通常能带来更好的训练效果和更快的收敛速度。

In [8]:
from tensorflow.keras.optimizers import SGD
optimizer = SGD(learning_rate=0.9, momentum=0.0, 
                nesterov=False)

net.compile(optimizer=optimizer,
            loss='sparse_categorical_crossentropy',
            metrics=['accuracy'])

In [9]:
net.fit(train_images, train_labels, 
        epochs=50, validation_split=0.1)

Epoch 1/50


I0000 00:00:1729518892.520666     101 service.cc:145] XLA service 0x79c3300075e0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1729518892.520719     101 service.cc:153]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
I0000 00:00:1729518892.520724     101 service.cc:153]   StreamExecutor device (1): Tesla T4, Compute Capability 7.5


  93/1688 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.0926 - loss: 2.5967

I0000 00:00:1729518894.103559     101 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


1688/1688 ━━━━━━━━━━━━━━━━━━━━ 7s 3ms/step - accuracy: 0.2943 - loss: 1.8291 - val_accuracy: 0.7438 - val_loss: 0.6584
Epoch 2/50
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.7590 - loss: 0.6113 - val_accuracy: 0.7287 - val_loss: 0.6689
Epoch 3/50
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.7977 - loss: 0.5202 - val_accuracy: 0.8143 - val_loss: 0.4766
Epoch 4/50
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.8211 - loss: 0.4662 - val_accuracy: 0.7977 - val_loss: 0.5110
Epoch 5/50
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.8295 - loss: 0.4410 - val_accuracy: 0.8385 - val_loss: 0.4257
Epoch 6/50
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.8348 - loss: 0.4244 - val_accuracy: 0.8025 - val_loss: 0.5049
Epoch 7/50
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.8383 - loss: 0.4193 - val_accuracy: 0.7983 - val_loss: 0.5245
Epoch 8/50
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.8248 - loss: 0.4516 - val_accurac

In [10]:
# 保存模型权重数据
net.save_weights("5.5_lenet.weights.h5")

In [11]:
net.evaluate(test_images, test_labels, verbose=2)

313/313 - 0s - 1ms/step - accuracy: 0.8586 - loss: 0.3684


[0.36839860677719116, 0.8586000204086304]

## 小结

* 卷积神经网络就是含卷积层的网络。
* LeNet交替使用卷积层和最大池化层后接全连接层来进行图像分类。